## Exploring the test data

One thing we need to do is investigate the test data and look for potential errors that could impact our score.

In [40]:
import pandas as pd
import numpy as np

In [41]:
# Load test data
df = pd.read_csv('./Data/test.csv')

df.head()

# So we can see there are some categorical variables, so we will need to encode then
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129050 entries, 0 to 129049
Data columns (total 10 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       129050 non-null  int64  
 1   carat    129050 non-null  float64
 2   cut      129050 non-null  object 
 3   color    129050 non-null  object 
 4   clarity  129050 non-null  object 
 5   depth    129050 non-null  float64
 6   table    129050 non-null  float64
 7   x        129050 non-null  float64
 8   y        129050 non-null  float64
 9   z        129050 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 9.8+ MB


In [42]:
# preprocessing
cut_labeling = {col: val+1 for val, col in enumerate(['Fair', 'Good', 'Very Good', 'Premium', 'Ideal'])}
color_labeling = {col: val+1 for val, col in enumerate(['J', 'I', 'H', 'G', 'F', 'E', 'D'])}
clarity_labeling = {col: val+1 for val, col in enumerate(
    ['I3', 'I2', 'I1', 'SI2', 'SI1', 'VVS2', 'VVS1', 'VS2', 'VS1', 'IF', 'FL'])}
# to preprocess label features, map label to integer number.
def preprocessing(df):
    df['cut'] = df['cut'].map(cut_labeling)
    df['color'] = df['color'].map(color_labeling)
    df['clarity'] = df['clarity'].map(clarity_labeling)
    return df

# Process data
df = preprocessing(df)

In [43]:
# We also know there are some 0 values in the data, so we will need to impute them
df[df==0].count()

id          0
carat       0
cut         0
color       0
clarity     0
depth       0
table       0
x           3
y           3
z          14
dtype: int64

In [45]:
# Make a copy of test data which only includes columns depth and table
df_copy = df[['id', 'depth', 'table']].copy()

# Find most similar data to those with ids_x in test_data
# We know that if x is missing then so to is y
ids_x = df[(df['x'] == 0)]['id']

# Loop through all the ids
for ix in ids_x.values:
    # Get the row and only keep the depth and table columns
    row = df[df['id'] == ix][['id', 'depth', 'table']]

    # Find the most similar data
    diff_df = df_copy[['depth', 'table']] - row[['depth', 'table']].values
    norm_df = diff_df.apply(lambda x: np.linalg.norm(x), axis=1)
    similar = df_copy.loc[norm_df.idxmin()]['id']

    # impute x and y of the similar data into the test data
    df.loc[df['id'] == ix, 'x'] = df[df['id'] == similar]['x'].values
    df.loc[df['id'] == ix, 'y'] = df[df['id'] == similar]['y'].values


# Using the formula for depth =  200 * z /(x+y), we can finally impute the z values
df['z'] =  df['depth'] * (df['x'] + df['y']) /200

In [37]:
inds = df['z'][df['z']==0].index
df.loc[inds, 'z'] = df['depth'][inds] * (df['x'][inds] + df['y'][inds]) /200

In [38]:
df.loc[inds, 'z'] = df['depth'][inds] * (df['x'][inds] + df['y'][inds]) /200

In [39]:
df.loc[inds]

,id,carat,cut,color,clarity,depth,table,x,y,z
344,193917,1.11,4,6,4,63.0,60.0,6.59,6.56,4.142250
2460,196033,1.10,3,6,4,62.5,60.0,6.52,6.57,4.090625
9036,202609,2.18,4,4,5,62.4,59.0,8.25,8.22,5.138640
10693,204266,2.20,4,2,9,59.4,59.0,8.49,8.45,5.031180
50969,244542,2.20,4,2,5,60.8,58.0,8.42,8.46,5.131520
66929,260502,2.20,4,2,5,61.2,59.0,8.43,8.39,5.146920
68406,261979,0.71,2,4,5,64.2,59.0,7.26,7.18,4.635240
72930,266503,2.18,4,3,5,59.1,61.0,8.47,8.41,4.988040
73937,267510,1.01,4,3,3,61.1,59.0,6.42,6.46,3.934840
74113,267686,0.71,3,5,4,64.1,56.0,4.23,4.27,2.724250
